In [1]:
pip install -r requirements.txt

                                              0.0/65.8 kB ? eta -:--:--
     ---------------------------------------- 65.8/65.8 kB 3.5 MB/s eta 0:00:00
                                              0.0/59.7 kB ? eta -:--:--
     ---------------------------------------- 59.7/59.7 kB 3.3 MB/s eta 0:00:00
                                              0.0/67.0 kB ? eta -:--:--
     ---------------------------------------- 67.0/67.0 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [6]:
from openai import OpenAI

client = OpenAI()

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4-1106-preview",
)
chat_completion.choices[0].message

ChatCompletionMessage(content='This is a test.', role='assistant', function_call=None, tool_calls=None)

In [16]:
from app.llm.utils import schema_to_openai_func


def _new_file(full_path: str) -> str:
    """
    Create a new file at the given path. Returns "success" if successful, otherwise an error message.
    """
    pass

def _write_file(full_path: str, content: str) -> str:
    """
    Write the given content to the given file. Returns "success" if successful, otherwise an error message.
    """

def _execute_command(command: str) -> str:
    """
    Execute the given command in the integrated terminal. Returns the output of the command.
    """

from pydantic import BaseModel, Field

class new_file(BaseModel):
    """
    Create a new file at the given path. Returns "success" if successful, otherwise an error message.
    """
    full_path: str = Field(description="The full path of the file to create.")
    
class write_file(BaseModel):
    """
    Write the given content to the given file. Returns "success" if successful, otherwise an error message.
    """
    full_path: str = Field(description="The full path of the file to write to.")
    content: str = Field(description="The content to write to the file.")

class execute_command(BaseModel):
    """
    Execute the given command in the integrated terminal. Returns the output of the command.
    """
    command: str = Field(description="The command to execute.")


In [38]:
from openai import OpenAI
client = OpenAI()

SYSTEM = """
You are an AI copilot directly integrated into an IDE. You have full access to the interface of the IDE by calling the provided function tools, such as creating new files, writing to files, and executing commands on the integrated terminal.
When the user requests something, you should call a sequence of appropriate functions to fulfill the request.
For example, when the user requests "make and run a simple hello world python program", you should call the following functions:
1. new_file("src/main.py")
2. write_file("src/main.py", "print('hello world')")
3. execute_command("python src/main.py")


"""

tools = [
  schema_to_openai_func(new_file),
  schema_to_openai_func(write_file),
  schema_to_openai_func(execute_command)
]
messages = [
    {"role": "system", "content": SYSTEM},
    {"role": "user", "content": "make and run a simple hello world python program"}
  ]
completion = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=messages,
  tools=tools,
  tool_choice="auto",
  temperature=0.0,
)

for tool_call in completion.choices[0].message.tool_calls:
    print(tool_call.function)

Function(arguments='{"full_path":"src/main.py"}', name='new_file')


In [36]:
for tool_call in completion.choices[0].message.tool_calls:
    print(tool_call.function)

Function(arguments='{"full_path":"src/hello_world.py"}', name='new_file')
